In [29]:
import pandas
import duckdb
from src.club_names import name_dict

In [2]:

players = pandas.read_csv(r"data/fifa/players_21.csv")
results = pandas.read_csv(r"data/results_premier/results.csv", encoding = "ISO-8859-1")



In [7]:
query = """
    SELECT 
      club_name
      , avg(overall) filter(where club_position = 'RES') as reserve_overall
      , avg(overall) filter(where club_position = 'SUB') as subs_overall
      , avg(overall) filter(where club_position not in ('RES','SUB') ) as titular_overall
      , avg(overall) filter(where club_position in ('ST','CAM','LS','LW', 'RW','CF','RS' , 'LM' , 'RF') ) as attack_overall
      , avg(overall) filter(where club_position in ( 'GK', 'LCB', 'RCB' ,'CDM', 'LDM' , 'RDM', 'LB', 'RB' ,'CB', 'RCB' , 'LCB' ) ) as defend_overall

    FROM players 
    where league_name = 'English Premier League' 
    group by club_name
    
"""
teams_21 = duckdb.query(query).to_df()


In [35]:

query = """
    SELECT 
        Season ,
        HomeTeam,  
        AwayTeam,
        FTHG as goals_home,
        FTAG as goals_away
        
    FROM results
    where 
        season = '2020-21'
"""
results_21 = duckdb.query(query).to_df()

In [38]:
results_21["away_code"] = results_21.AwayTeam.apply(lambda x: name_dict[x])
results_21["home_code"] = results_21.HomeTeam.apply(lambda x: name_dict[x])

In [45]:
query = """
    SELECT 
        home_code as home
        ,away_code as away
        ,goals_home
        ,goals_away
        , th.reserve_overall as home_reserve_overall
        , th.subs_overall as home_subs_overall
        , th.titular_overall as home_titular_overall
        , th.attack_overall as home_attack_overall
        , th.defend_overall as home_defend_overall
        , ta.reserve_overall as away_reserve_overall
        , ta.subs_overall as away_subs_overall
        , ta.titular_overall as away_titular_overall
        , ta.attack_overall as away_attack_overall
        , ta.defend_overall as away_defend_overall
    FROM results_21 r
    Left Join teams_21 th on r.home_code = th.club_name
    Left Join teams_21 ta on r.away_code = ta.club_name
"""
df = duckdb.query(query).to_df()

In [47]:
df

,home,away,goals_home,goals_away,home_reserve_overall,home_subs_overall,home_titular_overall,home_attack_overall,home_defend_overall,away_reserve_overall,away_subs_overall,away_titular_overall,away_attack_overall,away_defend_overall
0,Fulham,Arsenal,0,3,62.100000,74.000000,72.636364,73.000000,72.857143,72.300000,75.250000,80.818182,84.000000,80.250000
1,Crystal Palace,Southampton,1,0,61.400000,74.666667,77.090909,78.000000,76.600000,65.300000,71.333333,76.727273,77.333333,76.400000
2,Liverpool,Leeds United,4,3,70.500000,77.166667,87.454545,89.000000,87.333333,61.400000,68.500000,73.363636,72.000000,73.666667
3,West Ham United,Newcastle United,0,2,60.000000,74.666667,77.909091,77.333333,78.285714,66.500000,74.833333,77.727273,78.666667,78.750000
4,West Bromwich Albion,Leicester City,0,3,60.444444,70.500000,73.545455,73.666667,73.285714,70.100000,74.583333,80.363636,81.333333,79.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Liverpool,Crystal Palace,2,0,70.500000,77.166667,87.454545,89.000000,87.333333,61.400000,74.666667,77.090909,78.000000,76.600000
376,Manchester City,Everton,5,0,66.300000,79.166667,85.818182,85.333333,85.000000,68.300000,74.166667,80.181818,80.666667,80.500000
377,Sheffield United,Burnley,1,0,59.000000,70.916667,76.818182,74.500000,77.000000,59.571429,69.750000,77.363636,77.666667,78.200000
378,West Ham United,Southampton,3,0,60.000000,74.666667,77.909091,77.333333,78.285714,65.300000,71.333333,76.727273,77.333333,76.400000
